# Effects of Renewable and Conventional Electricity Generation on Spot Market Prices in Germany and Luxemburg

We use [data provided by the German Federal Network Agency](https://www.smard.de/home/downloadcenter/download-marktdaten) to investigate the effects of renewable and conventional electricity generation on spot market prices in the bidding zone Germany / Luxemburg from 2019 to 2021.
The data consists of hourly data on electricity generation in MWh separated by technology type (e.g. natural gas, solar, wind,...) and the intraday spot market prices as €/MWh values in 15-minute intervals.
We resample the latter to hourly intervals in order to simplify the analysis.

The results meet our expectation that renewables depress prices and conventionals cause higher prices due to the [merit-order](https://en.wikipedia.org/wiki/Merit_order).

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from tueplots import bundles
import os

In [ ]:
# plt.rcParams.update(bundles.neurips2021(usetex=False))

## Prepare Data for Bidding Zone DE/LU (2019-2021)
### Extract Prices

In [ ]:
src_path = '../dat/bundesnetzagentur'
price_files_2018_2021 = ['Grosshandelspreise_2018_2019.csv',
                         'Grosshandelspreise_2020_2021.csv']

prices_de_lu = pd.DataFrame()

for file in price_files_2018_2021:
    df = pd.read_csv(os.path.join(src_path, file),
                     delimiter=';',
                     # Use European decimal and thousands symbols
                     decimal=',', 
                     thousands='.',
                     # read '-' as np.nan
                     na_values=['-'],
                     parse_dates=[['Datum', 'Uhrzeit']])
    prices_de_lu = prices_de_lu.append(df)
    
prices_de_lu = prices_de_lu.set_index('Datum_Uhrzeit')
prices_de_lu = prices_de_lu.sort_index()

# Extract prices from 2019 onwards for bidding zone DE/LU
prices_de_lu = prices_de_lu.loc['2019':, 'Deutschland/Luxemburg[€/MWh]']
prices_de_lu

### Extract Electricity Generation Data

In [ ]:
erzeugung_in_regelzonen = os.path.join(src_path, 'erzeugung_in_regelzonen')
prod_files_de_lu = ['Realisierte_Erzeugung_DE_LU_2018-10-01_2019.csv',
                    'Realisierte_Erzeugung_DE_LU_2020_2021.csv']

prod_de_lu = pd.DataFrame()

for file in prod_files_de_lu:
    df = pd.read_csv(os.path.join(erzeugung_in_regelzonen, file),
                     delimiter=';',
                     # Use European decimal and thousands symbols, directly parse dates
                     decimal=',', 
                     thousands='.',
                     # read '-' as np.nan
                     na_values=['-'],
                     parse_dates=[['Datum', 'Uhrzeit']])
    prod_de_lu = prod_de_lu.append(df)

prod_de_lu = prod_de_lu.set_index('Datum_Uhrzeit')
prod_de_lu = prod_de_lu.sort_index()

# Extract production from 2019 onwards
prod_de_lu = prod_de_lu.loc['2019':]

# Resample to hours (same as prices)
prod_de_lu = prod_de_lu.resample('H').mean()

# Combine onshore and offshore wind
prod_de_lu['Wind'] = prod_de_lu.loc[:, ('Wind Offshore[MWh]', 'Wind Onshore[MWh]')].sum(axis=1)

### Join Data and Translate Columns

In [ ]:
combined = prod_de_lu.join(prices_de_lu)

column_names_map = {
    'Biomasse[MWh]': 'Biomass',
    'Wasserkraft[MWh]': 'Hydropower',
    'Wind Offshore[MWh]': 'Offshore Wind', 
    'Wind Onshore[MWh]': 'Onshore Wind',
    'Photovoltaik[MWh]': 'Solar',
    'Sonstige Erneuerbare[MWh]': 'Other Renewables',
    'Kernenergie[MWh]': 'Nuclear',
    'Braunkohle[MWh]': 'Brown Coal',
    'Steinkohle[MWh]': 'Black Coal',
    'Erdgas[MWh]': 'Natural Gas',
    'Pumpspeicher[MWh]': 'Pumped-Storage Hydro',
    'Sonstige Konventionelle[MWh]': 'Other Conventional Sources',
    'Deutschland/Luxemburg[€/MWh]': 'Price'
}

combined = combined.rename(columns=column_names_map)
combined['Solar & Wind'] = combined[['Solar', 'Wind']].sum(axis=1)
combined.index = combined.index.rename('time')

# some values are missing
combined = combined.dropna()

combined

## Some Visualizations

### Timeseries from August 2019
Shows diurnal variation of solar and negative prices due to high generation from wind.

In [ ]:
fig, ax1 = plt.subplots(figsize=(12,4))

ax2 = ax1.twinx()

date = '2019-08'

ax1.plot(combined.loc[date, 'Price'], color='black', label='Price')
ax1.set_ylabel('Spot Price [€/MWh]')

ax2.plot(combined.loc[date, 'Solar'], color='tab:orange', label='Solar')

ax2.plot(combined.loc[date, 'Wind'], color='tab:blue', label='Wind')
ax2.set_ylabel('Electricity Generation [MWh]')

handles, labels = [(a + b) for a, b in zip(ax1.get_legend_handles_labels(), ax2.get_legend_handles_labels())]
fig.legend(handles, labels, loc='upper center')

plt.plot()

### Quarterly Technology Mix

In [ ]:
quarterly       = combined.resample('3M').mean()
quarterly.index = quarterly.index.date

quarterly = quarterly.drop(columns=['Price', 'Solar & Wind', 'Wind', 'Pumped-Storage Hydro'])

fig = quarterly.plot(kind='bar',
              stacked=True,
              width=0.4,
              figsize=(12,6),
              rot=45)

fig.set_xlabel('Date')
fig.set_ylabel('Electricity Generation [MWh]')

plt.savefig('TestDataPresentation.pdf')
plt.show()

## Use Regression to Investigate Effect of Renewables and Conventionals on Spot Price

### Solar, Wind, and both

In [ ]:
cols = [('Solar', 'tab:orange'), ('Wind', 'tab:blue'), ('Solar & Wind', 'tab:green')]

fig, axs = plt.subplots(1, 3, figsize=(16,4), sharey=True)

axs[0].set_ylabel('Price [€/MWh]')

for i in range(0, len(axs)):
    col, color = cols[i]
    ax  = axs[i]
    
    X = combined[col].to_numpy().reshape(-1,1)
    y = combined['Price'].to_numpy()

    reg = LinearRegression()
    reg.fit(X, y)

    m = reg.coef_
    b = reg.intercept_
    
    ax.plot(X, y, 'o', color=color, ms=4, alpha=0.5)
    ax.plot(X, m*X + b, 'black')

    ax.set_xlabel(f'{col} Generation [MWh]')

### Comparison of Conventional and Renewable

In [ ]:
renewable = ['Biomass', 
             'Hydropower', 
             'Offshore Wind', 
             'Onshore Wind', 
             'Solar', 
             'Other Renewables']
conventional = ['Nuclear', 
                'Brown Coal', 
                'Black Coal',
                'Natural Gas',
                'Other Conventional Sources']

con_ren                 = combined # .resample('D').mean()
con_ren['Conventional'] = con_ren.loc[:, conventional].sum(axis=1)
con_ren['Renewable']    = con_ren.loc[:, renewable].sum(axis=1)
con_ren                 = con_ren.filter(['Conventional', 'Renewable', 'Price'])
con_ren['Total']        = con_ren.loc[:, ('Conventional', 'Renewable')].sum(axis=1)
con_ren

In [ ]:
X_con = con_ren['Conventional'].to_numpy().reshape(-1,1)
X_ren = con_ren['Renewable'].to_numpy().reshape(-1,1)
y     = con_ren['Price']

reg_con = LinearRegression()
reg_ren = LinearRegression()

reg_con.fit(X_con, y)
reg_ren.fit(X_ren, y)

m_con, b_con = reg_con.coef_, reg_con.intercept_
m_ren, b_ren = reg_ren.coef_, reg_ren.intercept_

plt.figure(figsize=(12,6))

ren_color = 'tab:olive'
con_color = 'tab:purple'

plt.plot(X_ren, y, 'o', label='Renewable', color=ren_color, ms=5, alpha=0.5)
plt.plot(X_con, y, 'o', label='Conventional', color=con_color, ms=5, alpha=0.5)

plt.plot(X_ren, m_ren * X_ren + b_ren, 'black', label='Regression Line (Renewable)', linewidth=3)
plt.plot(X_con, m_con * X_con + b_con, 'black', label='Regression Line (Conventional)', linewidth=3, linestyle='dotted')

plt.xlabel('Electricity Production [MWh]')
plt.ylabel('Price [€/MWh]')

plt.legend()
plt.show()

### Pumped-Storage Hydro Does Arbitrage in Time
Sort of an outlier: Pumped-Hydro does arbitrage in time by using electricity in times of low prices to pump water up and then releasing it in times of high prices to make a profit.
Hence, the regression line has a positive slope.

In [ ]:
X = combined['Pumped-Storage Hydro'].to_numpy().reshape(-1,1)
y = combined['Price'].to_numpy()

reg = LinearRegression()
reg.fit(X, y)

m = reg.coef_
b = reg.intercept_

plt.figure(figsize=(10,5))

plt.plot(X, y, 'o', ms=5, alpha=0.7)
plt.plot(X, m*X + b, 'r')

plt.xlabel('Pumped-Storage Hydro Electricity Generation [MWh]')
plt.ylabel('Price [€/MWh]')

# plt.tight_layout()
# plt.savefig('regression_test.pdf')
plt.show()